In [1]:
import sys
from itertools import islice, cycle, count
 
try:
    from itertools import compress
except ImportError:
    def compress(data, selectors):
        return (d for d, s in zip(data, selectors) if s)
 
 
def is_prime(n):
    return list(zip((True, False), decompose(n)))[-1][0]
 
class IsPrimeCached(dict):
    def __missing__(self, n):
        r = is_prime(n)
        self[n] = r
        return r

In [2]:
is_prime_cached = IsPrimeCached()
 
def croft():
    for p in (2, 3, 5):
        yield p
    roots = {9: 3, 25: 5}  # Map d**2 -> d.
    primeroots = frozenset((1, 7, 11, 13, 17, 19, 23, 29))
    selectors = (1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0)
    for q in compress(islice(count(7), 0, None, 2),cycle(selectors)):
        if q in roots:
            p = roots[q]
            del roots[q]
            x = q + 2*p
            while x in roots or (x % 30) not in primeroots:
                x += 2*p
            roots[x] = p
        else:
            roots[q*q] = q
            yield q
            
primes = croft
 
def decompose(n):
    res = []
    for p in primes():
        if p*p > n: break
        while n % p == 0:
            res.append(p)
            n //=p
    if n > 1:
        res.append(n)
    
    return res

In [3]:
def prod(arr):
    res = 1
    for elem in arr:
        res *= elem
    
    return res

def simplifyR(exp):
    a = decompose(exp[0])
    b = decompose(exp[1])
    i = 0
    while i < len(a):
        elem = a[i]
        if elem in b:
            b.pop(b.index(elem))
            a.pop(a.index(elem))
        else:
            i += 1

    if a == []:
        a = [1]
    elif b == []:
        b = [1]
        
    return [prod(a), prod(b)]

In [4]:
def sumR(arr):
    denom = 1
    for x in arr:
        denom *= x[1]
    
    s = 0
    for i in range(len(arr)):
        s += arr[i][0] * (denom//arr[i][1])
    
    return [s, denom]

def prodR(a,b):
    return simplifyR([a[0] * b[0], a[1] * b[1]])

def vprodR(arr):
    res = [1,1]
    for elem in arr:
        res = prodR(res, elem)
        
    return res

def divR(a,b):
    b = b[::-1]
    return prodR(a,b)

def subR(a,b):
    tmpA = [a[0] * b[1], a[1] * b[1]]
    tmpB = [b[0] * a[1], a[1] * b[1]]
    res = simplifyR([tmpA[0] - tmpB[0], tmpA[1]])
    return res

In [5]:
def geometricSum(r,a):
    return subR(prodR(a, [r[1], r[1] - r[0]]), a)

def getCommonDenominator(arr):
    res = []
    for elem in arr:
        decomposition = decompose(elem)
        for prime in list(set(decomposition)):
            if res.count(prime) < decomposition.count(prime):
                for i in range((decomposition.count(prime) - res.count(prime))):
                    res.append(prime)
                
    return prod(res)

def commonizeDenom(terminals):
    comDenom = getCommonDenominator([terminals[key][1] for key in terminals.keys()])    
    
    for key in terminals.keys():
        terminals[key] = [terminals[key][0] * (comDenom//terminals[key][1]), comDenom]
    
    return terminals, comDenom

In [81]:
import collections

class solver():
    def __init__(self, tansitionMatrix):
        self.tansitionMatrix = tansitionMatrix
        self.isTerminal = [True if sum(i) == 0 else False for i in tansitionMatrix]
        self.terminals = collections.defaultdict(lambda: [])
        self.handeled = [] 
        
    def visit(self, curIdx, curProb, visited = []):
        if visited.count(curIdx) > 0:
            if visited.count(curIdx) > 1:
                return

            cicle = visited[visited.index(curIdx):] + [curIdx]
            if cicle in self.handeled:
                return

            self.handeled.append(cicle)
            cicleProb = vprodR(curProb[visited.index(curIdx):])
            for i in range(len(self.tansitionMatrix[curIdx])):
                if self.tansitionMatrix[curIdx][i] != 0:
                    if self.isTerminal[i]:
                        termProb = vprodR(curProb[:visited.index(curIdx)] 
                                          + [[self.tansitionMatrix[curIdx][i], sum(self.tansitionMatrix[curIdx])]])
                        prob_i = geometricSum(r=cicleProb, a=termProb)
                        self.terminals[i] += [prob_i]
                        print(cicle, curProb)
                    else:
                        prob_i = curProb + [[self.tansitionMatrix[curIdx][i], sum(self.tansitionMatrix[curIdx])]]
                        self.visit(i, prob_i, visited+[curIdx])
            return

        if self.isTerminal[curIdx]:
            self.terminals[curIdx] += [vprodR(curProb)]
            return

        for i in range(0,len(self.tansitionMatrix[curIdx])):
            if self.tansitionMatrix[curIdx][i] != 0:
                prob_i = curProb + [[self.tansitionMatrix[curIdx][i], sum(self.tansitionMatrix[curIdx])]]

                self.visit(i, prob_i, visited+[curIdx])
        return 
    
    def getRes(self):
        self.visit(0, [], [])
        for key in self.terminals.keys():
            self.terminals[key] = simplifyR(sumR(self.terminals[key]))

        self.terminals, comDenom = commonizeDenom(self.terminals)

        res = []
        for i in range(len(self.tansitionMatrix)):
            if self.isTerminal[i]:
                if i in self.terminals.keys():
                    res.append(self.terminals[i][0])
                else:
                    res.append(0)

        res.append(comDenom)
        
        return res        


In [82]:
def prepareMatrix(m):
    for i in range(len(m)):
        if m[i][i] != 0:
            k = m[i][i]
            c = sum([1 if m[i][l] != 0 else 0 for l in range(len(m))]) - 1
            for j in range(len(m)):
                if j == i:
                    m[i][j] = 0
                else:
                    m[i][j] *= c + k
    return m

In [83]:
# tansitionMatrix = [[0, 2, 1, 0, 0],
#                    [0, 0, 0, 3, 4],
#                    [0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0]]

tansitionMatrix = [[0, 1, 0, 0, 0, 1],
                   [4, 0, 0, 3, 2, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]]

# tansitionMatrix = [[1, 1, 1], 
#                    [1, 1, 1], 
#                    [0, 0, 0]] 

# tansitionMatrix = [[1, 1, 1, 1], 
#                    [1, 1, 1, 1], 
#                    [1, 1, 1, 1],
#                    [0, 0, 0, 0]] 

# tansitionMatrix = [[0, 1, 0, 1], 
#                    [0, 0, 1, 0], 
#                    [0, 1, 0, 0],
#                    [0, 0, 0, 0]] 

m = prepareMatrix(tansitionMatrix)
s = solver(m)
res = s.getRes()
res

[1, 0, 1] [[1, 2], [4, 9], [1, 2]]
[1, 0, 1] [[1, 2], [4, 9], [1, 2]]
[0, 1, 0] [[1, 2], [4, 9]]


[0, 3, 2, 9, 14]

In [ ]:
sum(res[:-1])

In [16]:
prepareMatrix(tansitionMatrix)

[[0, 4, 4, 4], [4, 0, 4, 4], [4, 4, 0, 4], [0, 0, 0, 0]]

In [24]:
s = 0
for i in range(1,1000):
    s += (1/8)**i
    
print(s, 1/7)

0.14285714285714285 0.14285714285714285


In [86]:
from collections import defaultdict
from fractions import Fraction

def prod(arr):
    res = 1
    for elem in arr:
        res *= elem
    return res

def prepareMatrix(m):
    for i in range(len(m)):
        if m[i][i] != 0:
            k = m[i][i]
            c = sum([1 if m[i][l] != 0 else 0 for l in range(len(m))]) - 1
            for j in range(len(m)):
                if j == i:
                    m[i][j] = 0
                else:
                    m[i][j] *= c + k
    return m

class solver2():
    def __init__(self, m):
        self.m = m
        self.isTerminal = [True if sum(i) == 0 else False for i in self.m]
        self.terminals = defaultdict(lambda: [])
        self.handeled = [] 
        
    def visit(self, curIdx, curProb, visited = []):
            if visited.count(curIdx) > 0:
                if visited.count(curIdx) > 1:
                    return

                cicle = visited[visited.index(curIdx):] + [curIdx]
                if cicle in self.handeled:
                    return

                self.handeled.append(cicle)
                cicleProb = prod(curProb[visited.index(curIdx):])
                for i in range(len(self.m[curIdx])):
                    if self.m[curIdx][i] != 0:
                        if self.isTerminal[i]:
                            termProb = prod(curProb[:visited.index(curIdx)] 
                                              + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))])
                            prob_i = termProb * Fraction(cicleProb.denominator, cicleProb.denominator - cicleProb.numerator)
                            prob_i -= termProb
                            self.terminals[i] += [prob_i]
                            print(cicle,curProb)
                        else:
                            prob_i = curProb + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))]
                            self.visit(i, prob_i, visited+[curIdx])
                return
            
            if self.isTerminal[curIdx]:
                self.terminals[curIdx] += [prod(curProb)]
                return

            for i in range(0,len(self.m[curIdx])):
                if self.m[curIdx][i] != 0:
                    prob_i = curProb + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))]
                    self.visit(i, prob_i, visited+[curIdx])
            return 
        
    def getRes(self):
        self.visit(0, [], [])
        res = []
        for i in range(len(self.isTerminal)):
            if self.isTerminal[i]:
                if i in self.terminals.keys():
                    res.append(sum(self.terminals[i]))
                else:
                    res.append(Fraction(0,1))
        return res

In [89]:
# m = [[0, 1, 0], 
#      [0, 0, 0], 
#      [0, 0, 0]] 

m = [[0, 1, 0, 0, 0, 1],
   [4, 0, 0, 3, 2, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0]]

#m = prepareMatrix(m)
s = solver2(m)
res = s.getRes()
res

[1, 0, 1] [Fraction(1, 2), Fraction(4, 9), Fraction(1, 2)]
[1, 0, 1] [Fraction(1, 2), Fraction(4, 9), Fraction(1, 2)]
[0, 1, 0] [Fraction(1, 2), Fraction(4, 9)]


[Fraction(0, 1), Fraction(3, 14), Fraction(1, 7), Fraction(9, 14)]